<a href="https://colab.research.google.com/github/SoparAwayyy/Machine-Learning/blob/main/Week10/D2L.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DWI SAPUTRA SOPAR SIAGIAN

1103210220

D2l

In [ ]:
import pandas as pd
from d2l import torch as d2l
import pandas as pd
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np
import torch.nn as nn
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
%matplotlib inline
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from scipy.sparse import csr_matrix


In [ ]:
!pip install d2l # This command will install the d2l module, which seems to be required for your task.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Memuat dataset
file_path1 = '/content/drive/MyDrive/ML DataSet/Week10/test.csv'
file_path2 = '/content/drive/MyDrive/ML DataSet/Week10/train.csv'

dataset1 = pd.read_csv(file_path1)
dataset2 = pd.read_csv(file_path2)

In [ ]:
import pandas as pd
import torch
from torch import nn
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix
import numpy as np

# Memuat dataset
file_path1 = '/content/drive/MyDrive/ML DataSet/Week10/test.csv'
file_path2 = '/content/drive/MyDrive/ML DataSet/Week10/train.csv'

dataset1 = pd.read_csv(file_path1)
dataset2 = pd.read_csv(file_path2)

# Define features and target
features = dataset2.drop(columns=['Id', 'SalePrice'])
target = dataset2['SalePrice']

# Preprocessing Pipeline
numeric_features = features.select_dtypes(include=['int64', 'float64']).columns
categorical_features = features.select_dtypes(include=['object']).columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Neural Network Model
class Net(nn.Module):
    def __init__(self, input_dim):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Training function
def train_model(model, train_features, train_target, num_epochs, lr, batch_size):
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    train_loss = []

    # Konversi sparse matrix menjadi array NumPy
    if isinstance(train_features, csr_matrix):
        train_features = train_features.toarray()

    train_features = torch.tensor(train_features, dtype=torch.float32)
    train_target = torch.tensor(train_target.values, dtype=torch.float32).view(-1, 1)

    for epoch in range(num_epochs):
        for i in range(0, len(train_features), batch_size):
            inputs = train_features[i:i+batch_size]
            labels = train_target[i:i+batch_size]

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        train_loss.append(loss.item())

        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    return train_loss

# K-Fold Cross Validation and Model Selection
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
models = []
train_losses = []
val_losses = []

for fold, (train_idx, val_idx) in enumerate(kfold.split(features, target)):
    print(f'Fold {fold+1}')
    train_features, val_features = features.iloc[train_idx], features.iloc[val_idx]
    train_target, val_target = target.iloc[train_idx], target.iloc[val_idx]

    # Preprocess data
    train_features_preprocessed = preprocessor.fit_transform(train_features)
    val_features_preprocessed = preprocessor.transform(val_features)

    # Initialize model
    input_dim = train_features_preprocessed.shape[1]
    model = Net(input_dim)

    # Train model
    train_loss = train_model(model, train_features_preprocessed, train_target, num_epochs=100, lr=0.001, batch_size=32)
    train_losses.append(train_loss)
    models.append(model)

    # Validation
    val_features_preprocessed = torch.tensor(val_features_preprocessed.toarray(), dtype=torch.float32)
    val_outputs = model(val_features_preprocessed)
    val_loss = mean_squared_error(val_outputs.detach().numpy(), val_target.values)
    val_losses.append(val_loss)
    print(f'Validation Loss: {val_loss:.4f}')

# Mean Training Loss
mean_train_loss = np.mean(train_losses, axis=0)
print(f'Mean Training Loss: {mean_train_loss[-1]:.4f}')

# Best Model Selection based on Validation Loss
best_model_idx = np.argmin(val_losses)
best_model = models[best_model_idx]
print(f'Best Model Index: {best_model_idx}')

# Save Best Model
torch.save(best_model.state_dict(), 'best_model.pth')


In [ ]:
import matplotlib.pyplot as plt

# Plotting validation predictions vs actual values
plt.figure(figsize=(10, 6))
plt.scatter(val_outputs.detach().numpy(), val_target.values, color='blue', label='Actual vs Predicted')
plt.plot([min(val_target.values), max(val_target.values)], [min(val_target.values), max(val_target.values)], color='red', linestyle='--', label='Perfect Prediction')
plt.xlabel('Predicted Sale Price')
plt.ylabel('Actual Sale Price')
plt.title('Validation Predictions vs Actual Values')
plt.legend()
plt.grid(True)
plt.show()
